In [64]:
# !pip install -U sentence-transformers

In [65]:
import numpy as np
from scipy.spatial.distance import cosine, euclidean
from typing import List, Dict, Any

class VectorDatabase:
    def __init__(self):
        self.database = []
        
    def insert(self, entries: List[Dict[str, Any]]):
        # insert id, text and vector into the DB
        for entry in entries:
            self.database.append((entry['id'], entry['metadata']['text'], entry['vector']))

    def search(self, query_vector: np.ndarray, k=1, metric='cosine') -> List[str]:
        # Retreive entries closest to query
        distances = []
        for _, text, vector in self.database:
            if metric == 'cosine':
                distance = cosine(query_vector, vector)
            elif metric == 'dot':
                distance = np.dot(query_vector, vector)
            elif metric == 'euclidean':
                distance = euclidean(query_vector, vector)
            else:
                raise ValueError("Unsupported metric")
            distances.append((distance, text))
        
        # Sort usingdistance
        if metric in ['cosine', 'dot']:
            distances.sort(key=lambda x: x[0], reverse=True)
        else:
            distances.sort(key=lambda x: x[0])
            
        return [text for _, text in distances[:k]]

db = VectorDatabase()

In [66]:
import json

tokenizer = AutoTokenizer.from_pretrained("TaylorAI/bge-micro")
model = AutoModel.from_pretrained("TaylorAI/bge-micro")

# encode function using the embedding model
def encode(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.pooler_output.detach().numpy()[0]

#initialize db
with open('blog.json', 'r') as file: 
        blogs = json.load(file)
        entries = [{'id': blog['id'], 'metadata': blog['metadata'], 'vector': encode(blog['metadata']['text'])} for blog in blogs]
        db.insert(entries)
    
# query_vector = encode("What are the latest advancements in AI?")
# top_texts = db.search(query_vector, k=3, metric='cosine')
# print(top_texts)

In [67]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
import torch

class RAGSystem:
    def __init__(self, vector_database, model_name='gpt2'):
        self.vector_database = vector_database
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name)

    def generate_response(self, query, k=3, search_metric='cosine'):
        # Retrieve top k documents
        query_vector = encode(query)
        top_documents = self.vector_database.search(query_vector, k=k, metric=search_metric)
        
        # Create a prompt with query and context
        prompt = 'Given the context - '+ ",".join(top_documents) + ". " + query
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt')

        # Generate using GPT-2
        output_sequences = self.model.generate(input_ids, max_length=500, num_return_sequences=1, pad_token_id=self.tokenizer.eos_token_id)

        # Decode the generation
        generated_sequence = output_sequences[0]
        text = self.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        return text

def main():
    print("How can I help you today?")
    query = input()
    rag = RAGSystem(db)
    response = rag.generate_response(query)
    print(response)

if __name__ == "__main__":
    main()

How can I help you today?


 what are llms


Given the context - This journey is 1% finished: One of our favorite slogans at Facebook was always, “this journey is only 1% finished”. The same is true for AI in general: the world is only just discovering its power. As founders, we are humbled by the support from our amazing team of investors, as well as our core team of rock star engineers, scientists and others.,We have seen firsthand how AI can have a positive impact on the world, and how gratifying it is to help state-of-the-art AI research turn into something concrete that actually makes the world a better place. We are excited to continue that journey at Contextual AI.,Our Journey: In our journeys as research leaders at top AI institutions, both in industry (Microsoft Research, Meta/Facebook AI Research, Hugging Face) and academia (Cambridge, NYU, Stanford), we have always been attracted to the most difficult problems. We first started working together at Facebook in 2016 where we built a multimodal framework, synthesizing inf